In [1]:
# Load necessary libraries
import sys
import os

import copy
import random
import collections
from pathlib import Path

import pandas as pd
import numpy as np

import pormake
from pormake import *

import rodmof
from rodmof import *
from rodmof.rod_utils import *
from rodmof.rod_mof import RodTopology, RodBuildingBlock
pormake.log.disable_print()
pormake.log.disable_file_print()

>>> Console logs (under WARNING level) are disabled.


# 1. Load topologies and building blocks
Cautions:
    Need to insert the "index of vacancy edge for rod - rod connection" and the "index of rod"

In [2]:
## options
topo_path = "./rodmof/assets/topos_and_bbs/topologies/"
rod_bbs_path = "./rodmof/assets/topos_and_bbs/rodbbs/"

pormake_database = pormake.Database()

In [3]:
# Pre-define the indices for rod building block (rod_node_index) and the indices for connection between rod building blocks (rod_edge_index)
# rod_edge_index can be identified using view() function, it can be changed as the version of pormake
# So, highly recommend to check using following topology view() example!!, You can check via single generation example.
## Usage example:
## topology = pormake.Database().get_topo('cds')
## topology.view()

topo_data = dict()

topo_data['cds'] = dict(
    rod_edge_index = [2, 3], rod_node_index = [0]
)
topo_data['dia'] = dict(
    rod_edge_index = [8, 12, 20, 13, 21, 15, 9, 22], rod_node_index = [0]
)
topo_data['mog'] = dict(
    rod_edge_index = [14, 15, 16, 17], rod_node_index = [1]
)

In [4]:
# Show information of topologies
topology_names = list(topo_data.keys())

topologies = dict()
for n in topology_names:
    t = RodTopology(f'{topo_path}/{n}.cgd')
    t.add_rod_info(topo_data[n]['rod_edge_index'])
    topologies[n] = t

print(topologies)
print("%d topologies "%len(topologies))

{'cds': Topology cds, (4)-cn, num edge types: 1, 'dia': Topology dia, (4)-cn, num edge types: 1, 'mog': Topology mog, (4,4)-cn, num edge types: 2}
3 topologies 


In [5]:
rod_bbs = []

for bb in Path(rod_bbs_path).glob("*.xyz"):
    bb = RodBuildingBlock(bb)
    bb.has_metal = True
    indices = [int(i) for i in bb.name.split('_')[-2:]]
    bb.add_rod_info(indices)
    rod_bbs.append(bb)

In [6]:
'''
Regard porphyrin building block and "si" building block as organic building blocks
We remove specific normal building blocks in this works (ex. asymmetric linker)
'''

porphyrin_bb_names = [
    'N13',
]

si_bb_names = [
    "N96",
    "N601",
    "N585",
]

remove_bbs = [
    'N575', 'E106',  'E118', 'E157',
    'E183', 'E196', 'E26', 'E58', 'E6', 'E94',
    'N20', 'N353', 'N57', 'E114', 'E122', 'E15',
    'E192', 'E220', 'E42', 'E67', 'E76', 'N255', 
    'N468', 'N82', 'E116', 'E146', 'E171', 'E193', 
    'E232', 'E46', 'E69', 'E83', 'N160', 'N28', 'N488',
    'N99', 'N190', 'N484', 'N403',
]


# Remove assymetric linkers
asym_linkers_df = pd.read_csv("./rodmof/assets/asymmetric_linker_name.csv", header=None, index_col=0, sep=",")
asym_bbs = list(asym_linkers_df.index)

organic_bb_names = porphyrin_bb_names + si_bb_names
remove_bbs += asym_bbs
remove_bbs = list(set(remove_bbs))

In [7]:
node_bbs, edge_bbs = [], []

for bb_name in pormake_database.bb_list:
    bb = pormake_database.get_bb(bb_name)
    if bb_name.startswith('N'):
        if bb.name in organic_bb_names:
            bb.has_metal = False
        if bb.name in remove_bbs:
            bb.has_metal = True
        if bb.has_metal == False:
            node_bbs.append(bb)

    if bb_name.startswith('E'):
        if bb.name in remove_bbs:
            bb.has_metal = True
        if bb.has_metal == False:
            edge_bbs.append(bb)

In [8]:
name2bb = {'E0':None}
for bb in rod_bbs+node_bbs+edge_bbs:
    name2bb[bb.name] = bb

rod_names = [i.name for i in rod_bbs]
node_names = [i.name for i in node_bbs]
edge_names = [i.name for i in edge_bbs]  
print("Rod BBs : %d"%len(rod_bbs))
print("Normal Node BBs : %d"%len(node_bbs))
print("Edge BBs : %d\n"%len(edge_bbs))
print("Total BBs : %d"%len(name2bb))

Rod BBs : 19
Normal Node BBs : 103
Edge BBs : 143

Total BBs : 266


# 2. Match rod and node using local structure for each topology

This may helps for bulk generation. Initial screening of nodes and rods that cannot be fitted to the topology.

In [9]:
ROD_RMSD_CUT = 0.3
NODE_RMSD_CUT = 0.3

locator = pormake.Locator()

rod_seed = dict()
node_seed = dict()

for toponame in topology_names:
    topo = topologies[toponame]
    seed = []
    for local in topo.unique_local_structures:
        matched_bbs = []
        for bb in rod_bbs:
            if len(local.positions) == bb.n_connection_points:
                rmsd = locator.calculate_rmsd(local, bb)
                if rmsd < ROD_RMSD_CUT:
                    matched_bbs.append(bb.name)
        seed.append(matched_bbs)
    rod_seed[toponame] = seed.copy()

for toponame in topology_names:
    topo = topologies[toponame]
    seed = []
    for local in topo.unique_local_structures:
        matched_bbs = []
        for bb in node_bbs:
            if len(local.positions) == bb.n_connection_points:
                rmsd = locator.calculate_rmsd(local, bb)
                if rmsd < NODE_RMSD_CUT:
                    matched_bbs.append(bb.name)
        seed.append(matched_bbs)
    node_seed[toponame] = seed.copy()

/home/lim/anaconda3/envs/test/lib/python3.8/site-packages/pormake/locator.py:20: UserWarning: Optimal rotation is not uniquely or poorly defined for the given sets of vectors.
  U, rmsd = scipy.spatial.transform.Rotation.align_vectors(p, q)


# 3. MOF name generation
For efficient generation and estimation of database size, the names of MOFs are generated in advance of real generation.

In [12]:
## options
name_save_path = "./MOF_names.txt"

In [13]:
mof_names = []
mof_names_per_topology = dict()

for toponame in topology_names:
    count = 0
    topo = topologies[toponame]
    node_sites = [i for i in topo.unique_node_types \
                  if not i in topo_data[toponame]['rod_node_index']]
    if node_sites:
        node_candidates = node_seed[toponame][node_sites[0]]
    else:
        node_candidates = ['']

    rod_sites = [i for i in topo.unique_node_types \
                  if i in topo_data[toponame]['rod_node_index']]
    rod_candidates = rod_seed[toponame][rod_sites[0]]

    for r in rod_candidates:
        for c in node_candidates:
            if c:
                # Organic node exists -> Allow [E0:None] edge
                non_edge_names = ['E0'] + edge_names
                for e in non_edge_names:
                    namelist = [toponame] + [r] + [c] + [e]
                    name = '+'.join(namelist)
                    mof_names.append(name)
                    count += 1
            else:
                # No organic node -> Not allow [E0:None] edge
                for e in edge_names:
                    namelist = [toponame] + [r] + [c] + [e]
                    name = '+'.join(namelist)
                    mof_names.append(name)
                    count += 1

    mof_names_per_topology[toponame] = copy.deepcopy(count)

In [14]:
print(len(mof_names))

print(mof_names_per_topology)

39005
{'cds': 715, 'dia': 2002, 'mog': 36288}


In [15]:
f = open(name_save_path, "w")
for n in mof_names:
    f.write(n)
    f.write("\n")
    
f.close()

# 4. NAME to CIF framework
Convert name of mof into .cif file

In [27]:
## options
name_list_path = name_save_path
MOF_DIR = './Bulk_gen'

In [28]:
def name_to_mof(_mof_name, _topologies=topologies, _name2bb=name2bb):
    # MOF name
    tokens = _mof_name.split("+")
    _toponame = tokens[0]
    _rod_names  = [tokens[1]]
    _node_names = []
    _edge_names = []
    for bb in tokens[2:]:
        if bb.startswith("N"):
            _node_names.append(bb)
        if bb.startswith("E"):
            _edge_names.append(bb)

    # Topology
    _topology = _topologies[_toponame]

    # Rod BB
    _rod = [_name2bb[n] for n in _rod_names]
    
    # Node, Edge
    _node = [_name2bb[n] for n in _node_names]
    _edge = [_name2bb[n] for n in _edge_names]
    
    # NODE INSERT
    _all_nodes = {}
    for i in _topology.unique_node_types:
        if i in topo_data[_toponame]['rod_node_index']:
            _all_nodes[i] = _rod[0]
        else:
            _all_nodes[i] = _node[0]


    _builder = Builder()
    _bbs = _builder.make_bbs_by_type(_topology, _all_nodes)

    # EDGE INSERT
    for i in _topology.edge_indices:
        if i not in _topology.rod_edge_indices:
            _bbs[i] = _edge[0]

    # PERMUTATION FOR ROD-MOF
    if _rod[0].n_connection_points == 4:
        _node_slot = _topology.get_rod_permutation_info()
        _permutation = dict()
        for i in _node_slot.keys():
            if sum(_node_slot[i]) == 2:
                _permutation[i] = _rod[0].get_permutation_for_slot(_node_slot[i])
    else:
        _permutation = None
    
    _rod_connection_index = [np.where(_rod[0].connection_point_indices == i)[0][0] for i in _rod[0].rod_indices]

    _mof = _builder.build(_topology, _bbs, permutations=_permutation, rod_connection_indices=_rod_connection_index)
    return _mof

In [29]:
# helpers
def pick(list_like):
    return random.sample(list_like, 1)[0]

def count_normal_atoms(bb):
    if bb is None:
        return 0
    else:
        return np.sum(bb.atoms.get_chemical_symbols() != np.array("X"))

def calculate_n_atoms_of_rodmof(_mof_name, _topologies=topologies, _name2bb=name2bb):
    # MOF name
    tokens = _mof_name.split("+")
    _toponame = tokens[0]
    _rod_names  = [tokens[1]]
    _node_names = []
    _edge_names = []
    for bb in tokens[2:]:
        if bb.startswith("N"):
            _node_names.append(bb)
        if bb.startswith("E"):
            _edge_names.append(bb)

    # Topology
    _topology = _topologies[_toponame]    
    # Rod BB
    _rod = [_name2bb[n] for n in _rod_names]
    # Node, Edge
    _node = [_name2bb[n] for n in _node_names]
    _edge = [_name2bb[n] for n in _edge_names]

    # NODE INSERT
    _all_nodes = {}
    for i in _topology.unique_node_types:
        if i in topo_data[_toponame]['rod_node_index']:
            _all_nodes[i] = _rod[0]
        else:
            _all_nodes[i] = _node[0]
    
    nt_counts = {}
    for nt in _topology.unique_node_types:
        n_nt = np.sum(_topology.node_types == nt)
        nt_counts[nt] = n_nt

    et_counts = len(_topology.edge_indices) - len(_topology.rod_edge_indices)

    counts = 0
    for nt, bb in _all_nodes.items():
        counts += nt_counts[nt] * count_normal_atoms(bb)

    counts += et_counts * count_normal_atoms(_edge[0])
    return counts

In [30]:
lines = open(name_list_path,'r').readlines()
name_list = [line.rstrip() for line in lines]

ERROR_LOG = 'error_log.txt'

In [31]:
error_file = open(ERROR_LOG, 'w')

for i, name in enumerate(name_list):
    try:
        tokens = name.split("+")
        topo = tokens[0]

        # frameworks are classified as topology
        _MOF_DIR = MOF_DIR+f"/{topo}/"

        if not os.path.exists(_MOF_DIR):
            os.makedirs(_MOF_DIR)

        # exclude large frameworks (# of atoms > 1000) for convenience during molecular simulation
        n_atoms = calculate_n_atoms_of_rodmof(name)
        if n_atoms > 1000:
            continue

        mof = name_to_mof(name)
        new_mof = clean_rod_mof(mof)

        new_mof.write_cif(_MOF_DIR+name+'.cif')
    
    except Exception as e:
        print(name)
        print(e)
        print(name, file=error_file)
        
error_file.close()